In [1]:
import numpy as np


In [7]:
def dcg_score(y_true, y_score, *, k=None,
              log_base=2, sample_weight=None, ignore_ties=False):
    return np.average(_dcg_sample_scores(y_true, y_score, k=k, log_base=log_base,ignore_ties=ignore_ties),weights=sample_weight)

def _dcg_sample_scores(y_true, y_score, k=None,
                       log_base=2, ignore_ties=False):

    discount = 1 / (np.log(np.arange(y_true.shape[1]) + 2) / np.log(log_base))
    if k is not None:
        discount[k:] = 0
    if ignore_ties:
        ranking = np.argsort(y_score)[:, ::-1]
        ranked = y_true[np.arange(ranking.shape[0])[:, np.newaxis], ranking]
        cumulative_gains = discount.dot(ranked.T)
    else:
        discount_cumsum = np.cumsum(discount)
        cumulative_gains = [_tie_averaged_dcg(y_t, y_s, discount_cumsum) for y_t, y_s in zip(y_true, y_score)]
        cumulative_gains = np.asarray(cumulative_gains)
    return cumulative_gains

def _tie_averaged_dcg(y_true, y_score, discount_cumsum):
    _, inv, counts = np.unique( - y_score, return_inverse=True, return_counts=True)
    ranked = np.zeros(len(counts))
    np.add.at(ranked, inv, y_true)
    ranked /= counts
    groups = np.cumsum(counts) - 1
    discount_sums = np.empty(len(counts))
    discount_sums[0] = discount_cumsum[groups[0]]
    discount_sums[1:] = np.diff(discount_cumsum[groups])
    return (ranked * discount_sums).sum()

In [8]:
true_relevance = np.asarray([[10, 0, 0, 1, 5]])
# we predict scores for the answers
scores = np.asarray([[.1, .2, .3, 4, 70]])

In [9]:
dcg_score(true_relevance, scores)

9.499457825916874

---
## 분해 

In [13]:
y_true = true_relevance
y_score = scores
log_base = 2

In [14]:
discount = 1 / (np.log(np.arange(y_true.shape[1]) + 2) / np.log(log_base))

In [15]:
discount

array([1.        , 0.63092975, 0.5       , 0.43067656, 0.38685281])

In [18]:
y_true.shape

(1, 5)

In [26]:
np.log2(np.arange(y_true.shape[1]) + 2)

array([1.        , 1.5849625 , 2.        , 2.32192809, 2.5849625 ])

In [27]:
np.cumsum(discount)

array([1.        , 1.63092975, 2.13092975, 2.56160631, 2.94845912])

In [ ]:
_, inv, counts = np.unique( y_score, return_inverse=True, return_counts=True)
ranked = np.zeros(len(counts))
np.add.at(ranked, inv, y_true)
ranked /= counts
groups = np.cumsum(counts) - 1
discount_sums = np.empty(len(counts))
discount_sums[0] = discount_cumsum[groups[0]]
discount_sums[1:] = np.diff(discount_cumsum[groups])

In [28]:
_, inv, counts = np.unique( y_score, return_inverse=True, return_counts=True)

In [30]:
inv

array([0, 1, 2, 3, 4], dtype=int64)

In [31]:
counts

array([1, 1, 1, 1, 1], dtype=int64)

In [32]:
y_score

array([[ 0.1,  0.2,  0.3,  4. , 70. ]])

In [33]:
_

array([ 0.1,  0.2,  0.3,  4. , 70. ])

In [35]:
np.zeros(len(counts))

array([0., 0., 0., 0., 0.])

In [38]:
i = 0 
_, inv, counts = np.unique( y_score[i], return_inverse=True, return_counts=True)
ranked = np.zeros(len(counts))
np.add.at(ranked, inv, y_true[i])

In [39]:
ranked

array([10.,  0.,  0.,  1.,  5.])

In [40]:
inv

array([0, 1, 2, 3, 4], dtype=int64)

In [41]:
counts

array([1, 1, 1, 1, 1], dtype=int64)

In [43]:
y_score[i]

array([[ 0.1,  0.2,  0.3,  4. , 70. ]])

In [44]:
ranked

array([10.,  0.,  0.,  1.,  5.])

In [45]:
y_true[i]

array([10,  0,  0,  1,  5])